In [5]:
def alpha_od(G):
    """
    Compute the size of the maximum odd independent set in the graph G.
    """
    from sage.numerical.mip import MixedIntegerLinearProgram
    
    n = G.order()
    V = G.vertices()
    
    # Create the mixed integer linear program
    mip = MixedIntegerLinearProgram(maximization=True)
    
    # Define variables
    x = mip.new_variable(binary=True) # Indicator if vertex v is in the independent set
    y = mip.new_variable(binary=True) # Indicator if vertex v has neighbors in the independent set
    z = mip.new_variable(integer=True) # Counter for the vertex v
    
    # Objective function
    mip.set_objective(mip.sum(x[v] for v in V))
    
    # Constraints
    for u, v in G.edges(labels=False):
        mip.add_constraint(x[u] + x[v] <= 1)
        
    for u in V:
        sum_neighbors = mip.sum(x[v] for v in G.neighbors(u))
        mip.add_constraint(sum_neighbors <= n * y[u])
        mip.add_constraint(y[u] + sum_neighbors == 2*z[u])
    
    # Solve the MIP
    return mip.solve()

def chi_so(G):
    """
    Compute the size of the minimum strong odd coloring of the graph G.
    """
    from sage.numerical.mip import MixedIntegerLinearProgram
    
    n = G.order()
    V = G.vertices()
    
    # Create the mixed integer linear program
    mip = MixedIntegerLinearProgram(maximization=False)
    
    # Define variables
    x = mip.new_variable(binary=True) # Indicator if vertex v has color i
    y = mip.new_variable(binary=True) # Indicator if color i is used
    z = mip.new_variable(integer=True) # Counter for vertex v and color i
    w = mip.new_variable(binary=True) # Indicator if color i appears in the neighborhood of vertex v
    
    # Objective function
    mip.set_objective(mip.sum(y[i] for i in range(n))) # n is obvious upper bound on colors
    
    # Constraints
    for v in V:
        mip.add_constraint(mip.sum(x[v, i] for i in range(n)) == 1) # Each vertex gets exactly one color
        
    for u, v in G.edges(labels=False):
        for i in range(n):
            mip.add_constraint(x[u, i] + x[v, i] <= 1) # Adjacent vertices have different colors
            
    for i in range(n):
        mip.add_constraint(mip.sum(x[v, i] for v in V) <= n * y[i]) # If color i is used, y[i] = 1
   
    for v in V:
        for i in range(n):
            sum_neighbors = mip.sum(x[u, i] for u in G.neighbors(v))
            deg = len(G.neighbors(v))
            mip.add_constraint(w[v, i] <= sum_neighbors) # w[v,i] = 0 if color i does not appear in neighborhood of v
            mip.add_constraint(sum_neighbors <= deg * w[v, i]) # w[v,i] = 1 if color i appears in neighborhood of v
            # these two conditions together mean w[v,i] = 1 iff color i appears in neighborhood of v
            mip.add_constraint(w[v, i] + sum_neighbors == 2 * z[v, i]) # Strong odd coloring condition
            mip.add_constraint(z[v, i] >= 0)
    
    # Solve the MIP
    return mip.solve()

def generate_graphs_with_alpha_od_equal_to_1(n):
    """
    Iterate over all non-isomorphic graphs with n vertices using nauty_geng and return a set of those graphs G for which alpha_od(G) == 1. It is meant to used for n <= 9.
    """
    from sage.graphs.graph_generators import graphs
    
    matches = list()
    for G in graphs(n):
        if G.diameter() > 2:
            continue  # Neccessary condition for alpha_od(G) == 1 is diameter at most 2
        elif alpha_od(G) == 1:
            matches.append(G)
            
    return matches

# Example usage:
graphs_with_alpha_od_1 = generate_graphs_with_alpha_od_equal_to_1(3)
len(graphs_with_alpha_od_1)
# na mojem računalniku sem potreboval ~40 sekund za n=8

def is_claw_free(G):
    """
    Check if the given graph G is claw-free. A graph is claw-free if it does not contain K_{1,3} as an induced subgraph.
    """
    from sage.graphs.graph_generators import graphs
    
    claw = graphs.ClawGraph() # K_{1,3}
    return G.subgraph_search(claw, induced=True) is None # subgraph_search returns None if no such subgraph is found

def add_edges_until_diameter_leq_2(G, seed=None):
    """
    Given a graph G, iteratively add edges so that the resulting
    graph H has diameter <= 2.
    
    Process:
    - Compute the diameter path
    - Randomly pick two non-adjacent vertices on that path
    - Add one edge between them
    - Repeat
    
    Returns the modified graph H.
    """
    
    import random
    if seed is not None:
        random.seed(int(seed))

    H = G.copy()
    diameter = H.diameter()

    while diameter > 2:
        # Find all pairs of vertices with distance equal to diameter
        max_distance_pairs = []
        vertices = H.vertices()
        
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                u, v = vertices[i], vertices[j]
                if H.distance(u, v) == diameter:
                    max_distance_pairs.append((u, v))
        
        if not max_distance_pairs:
            break
            
        # Pick a random pair at maximum distance
        u, v = random.choice(max_distance_pairs)
        
        # Get the shortest path between them
        path = H.shortest_path(u, v)
        
        # Find non-adjacent vertex pairs in the path
        non_adjacent_pairs = []
        for i in range(len(path)):
            for j in range(i + 2, len(path)):  # i+2 ensures non-adjacent in path
                a, b = path[i], path[j]
                if not H.has_edge(a, b):
                    non_adjacent_pairs.append((a, b))
        
        if not non_adjacent_pairs:
            # Remove this pair and try another
            max_distance_pairs.remove((u, v))
            if max_distance_pairs:
                continue
            else:
                break
        
        # Randomly select a pair to connect
        a, b = random.choice(non_adjacent_pairs)
        
        H.add_edge(a, b)
        print(f"Added edge: ({a}, {b})")
        diameter = H.diameter()
    
    return H

def generate_large_graphs_with_alpha_od_equal_to_1(n, attempts, seed=None):
    """
    Generate 'count' random graphs with 'n' vertices and diameter <= 2,
    and return those graphs G for which alpha_od(G) == 1.
    """
    import random
    if seed is not None:
        random.seed(int(seed))
    
    matches = list()
    seen_isomorphs = set()
    
    for _ in range(attempts):
        G = graphs.RandomGNP(n, p=0.5, seed=random.randint(0, int(1e6)))
        H = add_edges_until_diameter_leq_2(G, seed=random.randint(0, int(1e6)))
        
        canon_repr = H.canonical_label().graph6_string()
        if canon_repr in seen_isomorphs:
            continue # dont compute alpha_od for already seen graphs
        
        if alpha_od(H) == 1:
            seen_isomorphs.add(canon_repr)
            matches.append(H)
            
    return matches



In [6]:
from sage.all import Graph
from sage.graphs.graph_generators import graphs

P4 = Graph(5)
C4 = Graph([(0,1),(1,2),(2,3),(3,0)])
C5 = Graph([(0,1),(1,2),(2,3),(3,4),(4,0)])

for name, G in [("P4", P4), ("C4", C4), ("C5", C5)]:
    val = alpha_od(G)
    print(name, "alpha_od =", val)

G1 = Graph(5)
G3 = graphs.CompleteBipartiteGraph(2, 3)
for name, G in [("G1", G1), ("G3", G3)]:
    val = chi_so(G)
    print(name, "chi_so =", val)

P4 alpha_od = 5.0
C4 alpha_od = 1.0
C5 alpha_od = 1.0
G1 chi_so = 1.0
G3 chi_so = 3.0


In [7]:
print(is_claw_free(graphs.CycleGraph(10)))
print(is_claw_free(graphs.PathGraph(7)))
print(is_claw_free(graphs.CompleteGraph(6)))

print(is_claw_free(graphs.StarGraph(4)))
print(is_claw_free(graphs.CompleteBipartiteGraph(3,3)))

True
True
True
False
False


In [8]:
G = graphs.PathGraph(5)
G.show(layout="circular")
print(G.diameter())
H = add_edges_until_diameter_leq_2(G, seed=42)
print(H.diameter())

H.show(layout="circular")

Graphics object consisting of 10 graphics primitives
4
Added edge: (0, 2)
Added edge: (1, 3)
Added edge: (0, 3)
2
Graphics object consisting of 13 graphics primitives


In [9]:
for G in generate_large_graphs_with_alpha_od_equal_to_1(30, 5, seed=123):
    G.show(layout="circular")

Graphics object consisting of 245 graphics primitives
Graphics object consisting of 229 graphics primitives
Graphics object consisting of 259 graphics primitives
Graphics object consisting of 258 graphics primitives
Graphics object consisting of 244 graphics primitives


In [10]:
from itertools import combinations
from sage.graphs.graph_generators import graphs


def prop_common_neighbor_at_least_two(G):
    """
    H1: Vsaka dva nesosednja vozlišča imata vsaj 2 skupna soseda.
    """
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not G.has_edge(u, v):
                if len(set(G.neighbors(u)).intersection(G.neighbors(v))) < 2:
                    return False
    return True

def prop_strong_overlap_distance_two(G):
    """
    H2: Za vsak par u,v z dist(u,v)=2 obstaja w v N(u)∩N(v)
        in w2 v N(u)∩N(v)∩N(w).
    """
    dist = G.distance_all_pairs()
    for u in G:
        for v in G:
            if u < v and dist[u][v] == 2:
                common1 = set(G.neighbors(u)).intersection(G.neighbors(v))
                if not common1:
                    return False
                found = False
                for w in common1:
                    common2 = (
                        set(G.neighbors(u))
                        .intersection(G.neighbors(v))
                        .intersection(G.neighbors(w))
                    )
                    if common2:
                        found = True
                        break
                if not found:
                    return False
    return True

def prop_no_induced_P4(G):
    """
    H3: G ne vsebuje induciranega P4.
    """
    P4 = graphs.PathGraph(4)
    V = G.vertices()
    for S in combinations(V, 4):
        H = G.subgraph(S)
        if H.is_isomorphic(P4):
            return False
    return True

def prop_no_induced_odd_cycles_ge_5(G):
    """
    H4: G ne vsebuje induciranih lihih ciklov dolžine >= 5 (C5, C7, C9).
    Za n <= 9 je to dovolj.
    """
    odd_lengths = [5, 7, 9]
    cycles = {k: graphs.CycleGraph(k) for k in odd_lengths if k <= G.order()}
    V = G.vertices()
    for k, Ck in cycles.items():
        for S in combinations(V, k):
            H = G.subgraph(S)
            if H.is_isomorphic(Ck):
                return False
    return True

def prop_min_degree_at_least_3(G):
    """
    H5: minimalna stopnja >= 3.
    """
    return min(G.degree()) >= 3


def prop_neighborhood_independence_at_most_2(G):
    """
    H6: Za vsak v velja, da α(G[N(v)]) <= 2.
    Implementacija brez uporabe independence_number():
    - v vsakem sosedstvu N(v) preverimo, ali obstaja neodvisna trojka {a,b,c}.
    - če taka trojka obstaja, lastnost pade (vrnemo False).
    """
    for v in G:
        Nv = list(G.neighbors(v))
        # če ima sosedstvo manj kot 3 vozlišča, ne more imeti neodvisne množice velikosti 3
        if len(Nv) < 3:
            continue

        # preverimo vse trojke v N(v)
        for a, b, c in combinations(Nv, 3):
            # neodvisna množica pomeni: ni nobenega roba med temi tremi
            if (not G.has_edge(a, b)
                and not G.has_edge(a, c)
                and not G.has_edge(b, c)):
                # našli smo neodvisno množico velikosti 3 v N(v)
                return False
    return True


def connected_graphs_upto(n_max):
    """
    Generator vseh povezanih, po izomorfizmu različnih grafov
    z n vozlišči, 1 <= n <= n_max.
    """
    for n in range(1, n_max + 1):
        for G in graphs.nauty_geng(f"{n} -c"):
            yield G

def counterexamples_necessity(prop, n_max):
    """
    NUJNOST: isce grafe z α_od(G) = 1, ki NE zadoščajo prop(G).

    Če vrne prazen seznam, velja
       α_od(G) = 1  ⇒  prop(G)
    za vse povezane grafe z največ n_max vozlišči.
    """
    bad = []
    for G in connected_graphs_upto(n_max):
        # opcijsko: pogoj diam(G) <= 2 pospeši, če je nujen
        if G.diameter() > 2:
            continue
        if alpha_od(G) == 1 and not prop(G):
            bad.append(G)
    return bad


def counterexamples_sufficiency(prop, n_max):
    """
    ZADOSTNOST: isce grafe, ki zadoščajo prop(G), a imajo α_od(G) != 1.

    Če vrne prazen seznam, velja
       prop(G)  ⇒  α_od(G) = 1
    za vse povezane grafe z največ n_max vozlišči.
    """
    bad = []
    for G in connected_graphs_upto(n_max):
        if prop(G) and alpha_od(G) != 1:
            bad.append(G)
    return bad

def stats_alpha_od_1_and_property(prop, n_max):
    """
    Vrne slovar:
      n ↦ (št. grafov z α_od(G)=1, št. tistih med njimi z lastnostjo prop)
    """
    stats = {}
    for n in range(1, n_max + 1):
        total_alpha1 = 0
        total_alpha1_P = 0
        for G in graphs.nauty_geng(f"{n} -c"):
            if G.diameter() > 2:
                continue
            if alpha_od(G) == 1:
                total_alpha1 += 1
                if prop(G):
                    total_alpha1_P += 1
        stats[n] = (total_alpha1, total_alpha1_P)
    return stats

properties = {
    "H1_common_neighbor_at_least_two": prop_common_neighbor_at_least_two,
    "H2_strong_overlap_distance_two": prop_strong_overlap_distance_two,
    "H3_no_induced_P4": prop_no_induced_P4,
    "H4_no_induced_odd_cycles_ge_5": prop_no_induced_odd_cycles_ge_5,
    "H5_min_degree_at_least_3": prop_min_degree_at_least_3,
    "H6_neighborhood_indep_le_2": prop_neighborhood_independence_at_most_2,
}

def run_all_tests(n_max=9):
    """
    Za vsako hipotezo H1,H2,H3,H4,H5,H6 izpiše:
      - št. kontra-primerov za nujnost,
      - št. kontra-primerov za zadostnost,
      - statistiko (koliko grafov z α_od=1 ima lastnost).
    """
    for name, prop in properties.items():
        print("="*70)
        print(f"Lastnost: {name}")
        print("-"*70)

        bad_nec = counterexamples_necessity(prop, n_max)
        bad_suf = counterexamples_sufficiency(prop, n_max)
        stats = stats_alpha_od_1_and_property(prop, n_max)

        print(f"Nujnost: št. kontra-primerov = {len(bad_nec)}")
        print(f"Zadostnost: št. kontra-primerov = {len(bad_suf)}")
        print("Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):")
        for n in sorted(stats.keys()):
            t1, t2 = stats[n]
            print(f"  n={n}: {t1}  /  {t2}")
        print()

In [11]:
run_all_tests(5)

Lastnost: H1_common_neighbor_at_least_two
----------------------------------------------------------------------


Nujnost: št. kontra-primerov = 8
Zadostnost: št. kontra-primerov = 2
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  3
  n=5: 11  /  5

Lastnost: H2_strong_overlap_distance_two
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 10
Zadostnost: št. kontra-primerov = 1
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  2
  n=5: 11  /  4

Lastnost: H3_no_induced_P4
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 3
Zadostnost: št. kontra-primerov = 5
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  8

Lastnost: H4_no_induced_odd_cycles_ge_5
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 1
Zadostnost: št. kontra-primero

In [12]:
def prop_R1_unique_common_neighbor_dist2(G):
    """
    R1: Za vsak par u,v z dist(u,v)=2 je |N(u) ∩ N(v)| = 1.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if dist[u][v] == 2:
                common = set(G.neighbors(u)).intersection(G.neighbors(v))
                if len(common) != 1:
                    return False
    return True

def prop_R2_alpha_square_le_2(G):
    """
    R2: α(G^2) <= 2, tj. v G^2 ni neodvisne trojke.
    To izvedemo tako, da preverimo vsak trojček (a,b,c):
    v G^2 ne sme biti trojček, kjer med nobenim parom ni roba.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    
    def adj_in_square(x, y):
        # povezava v G^2 ⇔ razdalja <= 2 in x != y
        if x == y:
            return False
        return dist[x][y] <= 2

    for a, b, c in combinations(V, 3):
        # preverimo, ali so a,b,c paroma nepovezani v G^2
        if (not adj_in_square(a, b) and
            not adj_in_square(a, c) and
            not adj_in_square(b, c)):
            # našli smo neodvisno trojko v G^2
            return False
    return True

def prop_R4_neighborhood_diameter_le_2(G):
    """
    R4: Za vsak v ima inducirani podgraf G[N(v)] premer <= 2
        (če ima vsaj 2 vozlisci).
    """
    for v in G:
        Nv = list(G.neighbors(v))
        if len(Nv) <= 1:
            continue  # premer ni smiseln / trivialen
        H = G.subgraph(Nv)
        # H je lahko nepovezan; v tem primeru je premer neskončen
        if not H.is_connected():
            return False
        if H.diameter() > 2:
            return False
    return True

def prop_D2_triangles_at_least_n(G):
    """
    D2: G ima vsaj |V(G)| trikotnikov.
    Trikotnik = 3 vozlišča, med katerimi so prisotni vsi 3 robovi.
    """
    V = list(G.vertices())
    n = len(V)
    triangles = 0
    for a, b, c in combinations(V, 3):
        if (G.has_edge(a, b) and
            G.has_edge(a, c) and
            G.has_edge(b, c)):
            triangles += 1
    return triangles >= n

def prop_D3_union_neighbors_covers_all(G):
    """
    D3: Za vsak par u,v velja |N(u) ∪ N(v)| >= n - 1.
    """
    V = list(G.vertices())
    n = len(V)
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            union = set(G.neighbors(u)).union(G.neighbors(v))
            if len(union) < n - 1:
                return False
    return True

def prop_L1_no_P3_in_neighborhoods(G):
    """
    L1: V nobenem sosedstvu N(v) ni induciranega P3.
    Tj. za vsak v in vsako trojko (a,b,c) v N(v) inducirani podgraf
    na {a,b,c} ni pot dolžine 2.
    """
    for v in G:
        Nv = list(G.neighbors(v))
        if len(Nv) < 3:
            continue
        for a, b, c in combinations(Nv, 3):
            # štejemo robove med a,b,c
            edges = 0
            if G.has_edge(a, b):
                edges += 1
            if G.has_edge(a, c):
                edges += 1
            if G.has_edge(b, c):
                edges += 1
            if edges == 2:
                # preverimo, da to res ni trikotnik (3 roba) in ne 1 rob
                # edges == 2 že pove, da je inducirani podgraf P3
                return False
    return True

def prop_L3_no_K2_plus_isolate_in_neighborhoods(G):
    """
    L3: V nobenem sosedstvu N(v) ni induciranega podgrafa tipa K2 + izoliran vrh,
        tj. na trojki {a,b,c} iz N(v) ne sme biti natanko 1 rob.
    """
    for v in G:
        Nv = list(G.neighbors(v))
        if len(Nv) < 3:
            continue
        for a, b, c in combinations(Nv, 3):
            edges = 0
            if G.has_edge(a, b):
                edges += 1
            if G.has_edge(a, c):
                edges += 1
            if G.has_edge(b, c):
                edges += 1
            if edges == 1:
                return False
    return True

def prop_L5_neighbor_degree_drop_at_most_1(G):
    """
    L5: Za vsak rob (u,v) velja deg(u) >= deg(v) - 1 in deg(v) >= deg(u) - 1.
    Upoštevamo, da G.edges() v Sage vrača trojke (u, v, label).
    """
    deg = {v: G.degree(v) for v in G.vertices()}
    
    for e in G.edges():
        # e je (u, v) ali (u, v, label)
        u, v = e[0], e[1]
        if deg[u] < deg[v] - 1:
            return False
        if deg[v] < deg[u] - 1:
            return False
    return True




In [13]:
properties2 = ({
    "R1_unique_common_neighbor_dist2": prop_R1_unique_common_neighbor_dist2,
    "R2_alpha_square_le_2": prop_R2_alpha_square_le_2,
    "R4_neighborhood_diam_le_2": prop_R4_neighborhood_diameter_le_2,
    "D2_triangles_at_least_n": prop_D2_triangles_at_least_n,
    "D3_union_neighbors_covers_all": prop_D3_union_neighbors_covers_all,
    "L1_no_P3_in_neighborhoods": prop_L1_no_P3_in_neighborhoods,
    "L3_no_K2_plus_isolate_in_neighborhoods": prop_L3_no_K2_plus_isolate_in_neighborhoods,
    "L5_neighbor_degree_drop_at_most_1": prop_L5_neighbor_degree_drop_at_most_1,
})

def run_all_tests(n_max=9):
    """
    Za vsako hipotezo H1,H2,H3,H4,H5,H6 izpiše:
      - št. kontra-primerov za nujnost,
      - št. kontra-primerov za zadostnost,
      - statistiko (koliko grafov z α_od=1 ima lastnost).
    """
    for name, prop in properties2.items():
        print("="*70)
        print(f"Lastnost: {name}")
        print("-"*70)

        bad_nec = counterexamples_necessity(prop, n_max)
        bad_suf = counterexamples_sufficiency(prop, n_max)
        stats = stats_alpha_od_1_and_property(prop, n_max)

        print(f"Nujnost: št. kontra-primerov = {len(bad_nec)}")
        print(f"Zadostnost: št. kontra-primerov = {len(bad_suf)}")
        print("Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):")
        for n in sorted(stats.keys()):
            t1, t2 = stats[n]
            print(f"  n={n}: {t1}  /  {t2}")
        print()


In [14]:
run_all_tests(5)

Lastnost: R1_unique_common_neighbor_dist2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 9
Zadostnost: št. kontra-primerov = 8
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  2
  n=5: 11  /  4

Lastnost: R2_alpha_square_le_2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 12
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11

Lastnost: R4_neighborhood_diam_le_2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 10
Zadostnost: št. kontra-primerov = 1
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  2
  n=5: 11  /  4

Lastnost: D2_triangles_at_least_n
--------------------------

In [15]:
def run_property_tests(prop, name, n_max=9):
    """
    Izvede enako analizo kot run_all_tests, ampak za eno konkretno lastnost 'prop'
    z imenom 'name'.
    """
    print("="*70)
    print(f"Lastnost (kombinacija): {name}")
    print("="*70)

    bad_nec = counterexamples_necessity(prop, n_max)
    bad_suf = counterexamples_sufficiency(prop, n_max)
    stats = stats_alpha_od_1_and_property(prop, n_max)

    print(f"Nujnost: št. kontra-primerov = {len(bad_nec)}")
    print(f"Zadostnost: št. kontra-primerov = {len(bad_suf)}")
    print("Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):")
    for n in sorted(stats.keys()):
        t1, t2 = stats[n]
        print(f"  n={n}: {t1}  /  {t2}")

    return bad_nec, bad_suf, stats

def prop_N3_neighborhood_connected(G):
    """
    N3: Za vsak v je G[N(v)] povezan (če ima N(v) vsaj 2 vozlisci).
    """
    for v in G:
        Nv = list(G.neighbors(v))
        if len(Nv) <= 1:
            continue
        H = G.subgraph(Nv)
        if not H.is_connected():
            return False
    return True

def prop_N4_dist2_has_common_neighbor(G):
    """
    N4: Za vsak par u,v z dist(u,v)=2 je |N(u) ∩ N(v)| >= 1.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if dist[u][v] == 2:
                common = set(G.neighbors(u)).intersection(G.neighbors(v))
                if len(common) == 0:
                    return False
    return True



In [16]:
def prop_candidate1(G):
    return (prop_R2_alpha_square_le_2(G)
            and prop_neighborhood_independence_at_most_2(G)
            and prop_no_induced_odd_cycles_ge_5(G))

def prop_candidate2(G):
    """
    candidate2 = candidate1 & N3:
      R2, H4, H12, plus soseske so povezane.
    """
    return (prop_candidate1(G)
            and prop_N3_neighborhood_connected(G))

def prop_candidate3(G):
    """
    candidate3 = candidate1 & N4:
      R2, H4, H12, plus vsak par na dist=2 ima skupnega soseda.
    """
    return (prop_candidate1(G)
            and prop_N4_dist2_has_common_neighbor(G))

In [17]:
bad_nec_c1, bad_suf_c1, stats_c1 = run_property_tests(prop_candidate1, "candidate1 = R2 & H4 & H12", n_max=5)
bad_nec_c2, bad_suf_c2, stats_c2 = run_property_tests(prop_candidate2, "candidate2 = candidate1 & N3", n_max=5)
bad_nec_c3, bad_suf_c3, stats_c3 = run_property_tests(prop_candidate3, "candidate3 = candidate1 & N4", n_max=5)

Lastnost (kombinacija): candidate1 = R2 & H4 & H12
Nujnost: št. kontra-primerov = 2
Zadostnost: št. kontra-primerov = 5
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  9
Lastnost (kombinacija): candidate2 = candidate1 & N3
Nujnost: št. kontra-primerov = 9
Zadostnost: št. kontra-primerov = 0
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  2
  n=5: 11  /  5
Lastnost (kombinacija): candidate3 = candidate1 & N4
Nujnost: št. kontra-primerov = 2
Zadostnost: št. kontra-primerov = 5
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  9


In [18]:
def exists_odd_path_length_3(G, u, v):
    # ali obstaja pot u - x - y - v, kjer so vsi robovi prisotni
    for x in G.neighbors(u):
        for y in G.neighbors(x):
            if y in G.neighbors(v):
                return True
    return False

def prop_M1_odd_complete(G):
    """
    M1: za vsak par u ≠ v obstaja liha pot med njima
    (tu: dolžine 1 ali 3, ker diam(G)=2).
    """
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if G.has_edge(u,v):
                # to je pot lihe dolžine 1
                continue
            # druga možnost: pot dolžine 3
            if exists_odd_path_length_3(G, u, v):
                continue
            return False
    return True


In [19]:
def prop_candidate5(G):
    return (prop_R2_alpha_square_le_2(G)
            and prop_M1_odd_complete(G))

In [20]:
bad_nec_c5, bad_suf_c5, stats_c5 = run_property_tests(prop_candidate5, "candidate5 = R2 & M1", 5)

Lastnost (kombinacija): candidate5 = R2 & M1
Nujnost: št. kontra-primerov = 2
Zadostnost: št. kontra-primerov = 3
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  3
  n=5: 11  /  11


Potencialne lastnosti

In [27]:
def prop_R2_alpha_square_le_2(G):
    """
    R2: α(G^2) <= 2, tj. v G^2 ni neodvisne trojke.
    To izvedemo tako, da preverimo vsak trojček (a,b,c):
    v G^2 ne sme biti trojček, kjer med nobenim parom ni roba.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    
    def adj_in_square(x, y):
        # povezava v G^2 ⇔ razdalja <= 2 in x != y
        if x == y:
            return False
        return dist[x][y] <= 2

    for a, b, c in combinations(V, 3):
        # preverimo, ali so a,b,c paroma nepovezani v G^2
        if (not adj_in_square(a, b) and
            not adj_in_square(a, c) and
            not adj_in_square(b, c)):
            # našli smo neodvisno trojko v G^2
            return False
    return True

def prop_M1_odd_complete(G):
    """
    M1: za vsak par u ≠ v obstaja liha pot med njima
    (tu: dolžine 1 ali 3, ker diam(G)=2).
    """
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if G.has_edge(u,v):
                # to je pot lihe dolžine 1
                continue
            # druga možnost: pot dolžine 3
            if exists_odd_path_length_3(G, u, v):
                continue
            return False
    return True

def prop_no_induced_odd_cycles_ge_5(G):
    """
    H4: G ne vsebuje induciranih lihih ciklov dolžine >= 5 (C5, C7, C9).
    Za n <= 9 je to dovolj.
    """
    odd_lengths = [5, 7, 9]
    cycles = {k: graphs.CycleGraph(k) for k in odd_lengths if k <= G.order()}
    V = G.vertices()
    for k, Ck in cycles.items():
        for S in combinations(V, k):
            H = G.subgraph(S)
            if H.is_isomorphic(Ck):
                return False
    return True

def prop_N4_dist2_has_common_neighbor(G):
    """
    N4: Za vsak par u,v z dist(u,v)=2 je |N(u) ∩ N(v)| >= 1.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if dist[u][v] == 2:
                common = set(G.neighbors(u)).intersection(G.neighbors(v))
                if len(common) == 0:
                    return False
    return True

def prop_H12_prime_deg_ge_3_neigh_indep_le_2(G):
    """
    H12': Za vsako oglišče v s stopnjo >= 3 velja, da v sosedstvu N(v)
    ni neodvisne množice velikosti 3.

    Formalno:
      za vsak v z deg(v) >= 3 je α(G[N(v)]) <= 2.
    Implementirano z brute-force preverjanjem vseh trojk v N(v).
    """
    for v in G:
        # pogoj postavimo samo za oglišča z dovolj veliko stopnjo
        if G.degree(v) < 3:
            continue

        Nv = list(G.neighbors(v))
        # če ima N(v) manj kot 3 oglišča, ne more imeti neodvisne trojke
        if len(Nv) < 3:
            continue

        # preverimo vse trojke v N(v)
        for a, b, c in combinations(Nv, 3):
            # neodvisna trojka: med a,b,c ni nobenega roba
            if (not G.has_edge(a, b)
                and not G.has_edge(a, c)
                and not G.has_edge(b, c)):
                # našli smo neodvisno trojko v N(v) -> lastnost pade
                return False

    # če pri nobenem v nismo našli neodvisne trojke, lastnost velja
    return True

propertiesuporabni = ({
    "R2_alpha_square_le_2": prop_R2_alpha_square_le_2,
    "M1": prop_M1_odd_complete,
    "H4": prop_no_induced_odd_cycles_ge_5,
    "N4": prop_N4_dist2_has_common_neighbor,
    "H12": prop_H12_prime_deg_ge_3_neigh_indep_le_2
})

def run_all_tests(n_max=9):
    """
    Za vsako hipotezo H1,H2,H3,H4,H5,H6 izpiše:
      - št. kontra-primerov za nujnost,
      - št. kontra-primerov za zadostnost,
      - statistiko (koliko grafov z α_od=1 ima lastnost).
    """
    for name, prop in propertiesuporabni.items():
        print("="*70)
        print(f"Lastnost: {name}")
        print("-"*70)

        bad_nec = counterexamples_necessity(prop, n_max)
        bad_suf = counterexamples_sufficiency(prop, n_max)
        stats = stats_alpha_od_1_and_property(prop, n_max)

        print(f"Nujnost: št. kontra-primerov = {len(bad_nec)}")
        print(f"Zadostnost: št. kontra-primerov = {len(bad_suf)}")
        print("Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):")
        for n in sorted(stats.keys()):
            t1, t2 = stats[n]
            print(f"  n={n}: {t1}  /  {t2}")
        print()

In [22]:
run_all_tests(5)

Lastnost: R2_alpha_square_le_2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 12
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11

Lastnost: M1
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 2
Zadostnost: št. kontra-primerov = 3
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  3
  n=5: 11  /  11

Lastnost: H4
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 1
Zadostnost: št. kontra-primerov = 12
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  10

Lastnost: N4
----------------------------------------------------------------------
Nujnost: št. kontra-primer

In [23]:
run_all_tests(6)

Lastnost: R2_alpha_square_le_2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 79
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11
  n=6: 43  /  43

Lastnost: M1
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 3
Zadostnost: št. kontra-primerov = 34
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  3
  n=5: 11  /  11
  n=6: 43  /  42

Lastnost: H4
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 6
Zadostnost: št. kontra-primerov = 79
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  10
  n=6: 43  /  38

Lastnost: N4
---------------------------------------------

In [24]:
run_all_tests(7)

Lastnost: R2_alpha_square_le_2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 638
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11
  n=6: 43  /  43
  n=7: 266  /  266

Lastnost: M1
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 7
Zadostnost: št. kontra-primerov = 349
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  3
  n=5: 11  /  11
  n=6: 43  /  42
  n=7: 266  /  262

Lastnost: H4
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 68
Zadostnost: št. kontra-primerov = 600
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  10
  n=6: 43  /  38
  n=7: 266  /  20

In [25]:
run_all_tests(8)

Lastnost: R2_alpha_square_le_2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 8208
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11
  n=6: 43  /  43
  n=7: 266  /  266
  n=8: 3042  /  3042

Lastnost: M1
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 28
Zadostnost: št. kontra-primerov = 5405
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  1
  n=4: 4  /  3
  n=5: 11  /  11
  n=6: 43  /  42
  n=7: 266  /  262
  n=8: 3042  /  3021

Lastnost: H4
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 1301
Zadostnost: št. kontra-primerov = 6607
Statistika po n (n: α_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n